In [9]:
!pip install flickrapi
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
dfraw= pd.read_csv('../data/raw/df_with_picStatus.csv')

/var/folders/gl/_lkmfl_d3_sbkbd9f1zkt79h0000gq/T/ipykernel_44342/1410185032.py:3: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  dfraw= pd.read_csv('../data/raw/df_with_picStatus.csv')


In [7]:
urls= dfraw['url'].to_list()
urls[0]

'https://www.flickr.com/photos/58708830@N00/385070026'

In [8]:
len(urls)

304952

In [10]:
import flickrapi
import requests
import threading
import os
import concurrent.futures
from tqdm import tqdm
from joblib import Parallel, delayed  # Import joblib

In [6]:
urls[0]

'https://www.flickr.com/photos/58708830@N00/385070026'

# fixed code

In [12]:
import flickrapi
import requests
import os
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize the Flickr API with your API key and secret
api_key = '747002b66324f9570ea294e380992a9d'
api_secret = '74019702dd3b6de2'
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

# Path to save downloaded images
download_path = r'D:\old\data255_deeplearning_team2\data\raw\images'
if not os.path.exists(download_path):
    os.makedirs(download_path)

# Function to download an image
def download_image(url, download_path):
    try:
        # Extract the user ID and photo ID from the URL
        parts = url.strip().split('/')
        user_id = parts[-2].split('@')[0]
        photo_id = parts[-1]
        
        # Check if image already exists
        image_path = os.path.join(download_path, f"{photo_id}.jpg")
        if os.path.exists(image_path):
            logger.info(f"Image with ID {photo_id} already downloaded.")
            return None, 'downloaded already'
        
        # Get the photo info using the Flickr API
        photo_info = flickr.photos.getInfo(photo_id=photo_id, user_id=user_id)
        
        # Extract the image URL
        farm_id = photo_info['photo']['farm']
        server_id = photo_info['photo']['server']
        id = photo_info['photo']['id']
        secret = photo_info['photo']['secret']
        format = 'jpg'  # Assuming jpg format, you can modify this if needed
        image_url = f"https://farm{farm_id}.staticflickr.com/{server_id}/{id}_{secret}.{format}"
        
        # Download and save the image
        response = requests.get(image_url, stream=True)
        with open(image_path, 'wb') as img_file:
            for chunk in response.iter_content(chunk_size=8192):
                img_file.write(chunk)
        logger.info(f"Downloaded image with ID {photo_id}")
        return None, url
    except Exception as e:
        logger.error(f"Error while processing {photo_id}: {str(e)}")
        return str(e), url

In [16]:
# Download images using multithreading
results = Parallel(n_jobs=25)(
    delayed(download_image)(url, download_path) for url in tqdm(urls[:1], desc="Downloading images")
)

In [13]:
# Download images using multithreading
results = Parallel(n_jobs=25)(
    delayed(download_image)(url, download_path) for url in tqdm(urls[2500:3000], desc="Downloading images")
)

In [17]:
results

[(None, 'https://www.flickr.com/photos/58708830@N00/385070026')]

In [11]:
# Separate out errors and error-causing URLs
errors, error_urls = zip(*results)
error_urls = [url for err, url in zip(errors, error_urls) if err is not None]

# Print any errors that occurred during the download
if error_urls:
    logger.error("Errors occurred during the download:")
    for url in error_urls:
        logger.error(url)
else:
    logger.info("All images downloaded successfully!")

ERROR:__main__:Errors occurred during the download:
ERROR:__main__:https://www.flickr.com/photos/47649296@N06/16575004613
ERROR:__main__:https://www.flickr.com/photos/131575265@N07/16575571554
ERROR:__main__:https://www.flickr.com/photos/131747546@N05/16577791947
ERROR:__main__:https://www.flickr.com/photos/131747546@N05/16577792487
ERROR:__main__:https://www.flickr.com/photos/131747546@N05/16577798037
ERROR:__main__:https://www.flickr.com/photos/131747546@N05/16577798567
ERROR:__main__:https://www.flickr.com/photos/129754178@N07/16577964174
ERROR:__main__:https://www.flickr.com/photos/131797077@N06/16578524368
ERROR:__main__:https://www.flickr.com/photos/125141587@N02/16578909449
ERROR:__main__:https://www.flickr.com/photos/92009365@N03/16579426543
ERROR:__main__:https://www.flickr.com/photos/92009365@N03/16579427073
ERROR:__main__:https://www.flickr.com/photos/85743674@N00/16584964297
ERROR:__main__:https://www.flickr.com/photos/77706245@N08/16585154084
ERROR:__main__:https://www.fli

In [15]:
len(error_urls)

68

In [17]:
len(os.listdir(download_path))

432

In [18]:
432+68

500

# doing it for all the urls

In [19]:
# Download images using multithreading
results = Parallel(n_jobs=60)(
    delayed(download_image)(url, download_path) for url in tqdm(urls, desc="Downloading images")
)

In [20]:
251295/304952 

0.824047718985283

In [18]:
dfraw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304952 entries, 0 to 304951
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   url                   304952 non-null  object 
 1   label                 304952 non-null  float64
 2   Category              304952 non-null  object 
 3   Concept               304952 non-null  object 
 4   Pid                   304952 non-null  int64  
 5   Uid                   304952 non-null  object 
 6   Subcategory           304952 non-null  object 
 7   Alltags               304952 non-null  object 
 8   Title                 294408 non-null  object 
 9   Postdate              304952 non-null  int64  
 10  Longitude             304952 non-null  float64
 11  Geoaccuracy           304952 non-null  int64  
 12  Latitude              304952 non-null  float64
 13  photo_firstdatetaken  304952 non-null  int64  
 14  location_description  152478 non-null  object 
 15  

In [21]:
import os
os.chdir(r'D:\old\data255_deeplearning_team2\data\raw\images')
print(len(os.listdir()))

251295


In [22]:
dfraw[dfraw['url'].str.contains('385070026')].to_dict()

{'url': {0: 'https://www.flickr.com/photos/58708830@N00/385070026'},
 'label': {0: 11.18},
 'Category': {0: 'Fashion'},
 'Concept': {0: 'glam'},
 'Pid': {0: 775},
 'Uid': {0: '59@N75'},
 'Subcategory': {0: 'Fashion'},
 'Alltags': {0: 'rock punk transgender tranny electronicmusic electro glam electronica luisdrayton fusionrecords thefusionnetwork lmwcphotography'},
 'Title': {0: 'Luis Drayton - Edinburgh shoot #6'},
 'Postdate': {0: 1446016778},
 'Longitude': {0: 0.0},
 'Geoaccuracy': {0: 0},
 'Latitude': {0: 0.0},
 'photo_firstdatetaken': {0: 1904010100},
 'location_description': {0: '0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [23]:
dfraw['img']= dfraw['url'].apply(lambda x: x.split('/')[-1]+'.jpg')
dfraw

,url,label,Category,Concept,Pid,Uid,Subcategory,Alltags,Title,Postdate,...,location_description,photo_count,ispro,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,Pathalias,img
0,https://www.flickr.com/photos/58708830@N00/385...,11.18,Fashion,glam,775,59@N75,Fashion,rock punk transgender tranny electronicmusic e...,Luis Drayton - Edinburgh shoot #6,1446016778,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",6828,1,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830.0,1,luisdrayton,385070026.jpg
1,https://www.flickr.com/photos/97042891@N00/943...,15.15,Travel&Active&Sports,fifa,1075,1@N18,Soccer,brazil rio brasil riodejaneiro by maria fifa m...,Arena da Barra - Arena HSBC - Arena do PAN #...,1454983379,...,NaN,32102,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743.0,0,claudiolara,943750056.jpg
2,https://www.flickr.com/photos/18583731@N07/324...,10.99,Entertainment,cinema,4890,351@N64,Movies,old cinema beauty marilyn photoshop movie joke...,MARILYN 2015,1433118604,...,"0.0334692,-0.0585118,0.158872,-0.000377208,-0....",276,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600.0,-3,rizzato,3246928439.jpg
3,https://www.flickr.com/photos/98216376@N00/343...,8.63,Holiday&Celebrations,old,6568,6@N59,Birthday,pictures old family scans brothers sister 1958...,Knikkertijd - 1959,1451577600,...,NaN,32102,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743.0,0,4kleuren,3432316502.jpg
4,https://www.flickr.com/photos/35797910@N08/353...,11.16,Food,thirsty,7079,1617@N40,Drinks,hot sahara animal animals desert bottles drink...,CAMELS01,1425744438,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",32102,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743.0,0,glosackmd,3538960847.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304947,https://www.flickr.com/photos/120541745@N06/27...,4.00,Animal,creature,1118328,62225@N20,Insects,york travel sky urban sun france color tree gr...,Mushrooms,1433577362,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",8301,1,"0.104426,0.0598804,-0.0149883,0.0338535,-0.007...",0,49,1277859911.0,12,None,27597418605.jpg
304948,https://www.flickr.com/photos/92475605@N03/275...,1.00,Social&People,portrait,1118334,108435@N35,Selfies,portrait cute animal cat furry panasonic calic...,Evie,1439629953,...,NaN,801,0,"0.114495,-0.0303421,-0.0893819,0.0774031,-0.08...",0,3,1398624531.0,-6,matty3126,27597517425.jpg
304949,https://www.flickr.com/photos/43770052@N08/275...,2.00,Travel&Active&Sports,exotic,1118342,84730@N71,Cars,car exotic german malaysia kualalumpur superca...,BSSLS+R8GT!!,1444462129,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",574,0,"0.18969,-0.0675743,-0.125082,0.097416,-0.02100...",0,2,1394377573.0,0,khooxotics,27597606545.jpg
304950,https://www.flickr.com/photos/50969511@N02/275...,6.89,Whether&Season,cloud,1118344,9542@N37,Cloudy,city newzealand christchurch cloud building ca...,C to A1,1442458947,...,NaN,7167,1,"0.142518,-0.0121231,-0.0464556,0.0887664,-0.02...",0,20,1234576422.0,10,stevetaylors,27597637235.jpg


In [24]:
dfraw['img_access']= dfraw['img'].apply(lambda x: 1 if os.path.exists(x) else 0)
dfraw['img_access'].value_counts()

1    251295
0     53657
Name: img_access, dtype: int64

In [25]:
dfraw.columns

Index(['url', 'label', 'Category', 'Concept', 'Pid', 'Uid', 'Subcategory',
       'Alltags', 'Title', 'Postdate', 'Longitude', 'Geoaccuracy', 'Latitude',
       'photo_firstdatetaken', 'location_description', 'photo_count', 'ispro',
       'user_description', 'canbuypro', 'timezone_timezone_id',
       'photo_firstdate', 'timezone_offset', 'Pathalias', 'img', 'img_access'],
      dtype='object')

In [26]:
dfraw= dfraw[['url', 'img', 'img_access', 'label', 'Category', 'Concept', 'Pid', 'Uid', 'Subcategory',
       'Alltags', 'Title', 'Postdate', 'Longitude', 'Geoaccuracy', 'Latitude',
       'photo_firstdatetaken', 'location_description', 'photo_count', 'ispro',
       'user_description', 'canbuypro', 'timezone_timezone_id',
       'photo_firstdate', 'timezone_offset', 'Pathalias']]
dfraw.to_csv('../df_with_picStatus.csv', index=False)

In [11]:
df= dfraw[dfraw['img_access']==1]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 251295 entries, 0 to 304951
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   url                   251295 non-null  object 
 1   img                   251295 non-null  object 
 2   img_access            251295 non-null  int64  
 3   label                 251295 non-null  float64
 4   Category              251295 non-null  object 
 5   Concept               251295 non-null  object 
 6   Pid                   251295 non-null  int64  
 7   Uid                   251295 non-null  object 
 8   Subcategory           251295 non-null  object 
 9   Alltags               251295 non-null  object 
 10  Title                 242528 non-null  object 
 11  Postdate              251295 non-null  int64  
 12  Longitude             251295 non-null  float64
 13  Geoaccuracy           251295 non-null  int64  
 14  Latitude              251295 non-null  float64
 15  

# group Info DF

In [12]:
df['imgID']= df['img'].apply(lambda x: x.split('.')[0])
df

/var/folders/gl/_lkmfl_d3_sbkbd9f1zkt79h0000gq/T/ipykernel_44342/1539463631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['imgID']= df['img'].apply(lambda x: x.split('.')[0])


,url,img,img_access,label,Category,Concept,Pid,Uid,Subcategory,Alltags,...,location_description,photo_count,ispro,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,Pathalias,imgID
0,https://www.flickr.com/photos/58708830@N00/385...,385070026.jpg,1,11.18,Fashion,glam,775,59@N75,Fashion,rock punk transgender tranny electronicmusic e...,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",6828,1,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830.0,1,luisdrayton,385070026
1,https://www.flickr.com/photos/97042891@N00/943...,943750056.jpg,1,15.15,Travel&Active&Sports,fifa,1075,1@N18,Soccer,brazil rio brasil riodejaneiro by maria fifa m...,...,NaN,32102,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743.0,0,claudiolara,943750056
2,https://www.flickr.com/photos/18583731@N07/324...,3246928439.jpg,1,10.99,Entertainment,cinema,4890,351@N64,Movies,old cinema beauty marilyn photoshop movie joke...,...,"0.0334692,-0.0585118,0.158872,-0.000377208,-0....",276,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600.0,-3,rizzato,3246928439
4,https://www.flickr.com/photos/35797910@N08/353...,3538960847.jpg,1,11.16,Food,thirsty,7079,1617@N40,Drinks,hot sahara animal animals desert bottles drink...,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",32102,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743.0,0,glosackmd,3538960847
5,https://www.flickr.com/photos/35797910@N08/354...,3544037429.jpg,1,12.74,Travel&Active&Sports,flip,7111,1617@N40,Gymnastics,street people india portraits photography delh...,...,NaN,32102,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743.0,0,glosackmd,3544037429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304947,https://www.flickr.com/photos/120541745@N06/27...,27597418605.jpg,1,4.00,Animal,creature,1118328,62225@N20,Insects,york travel sky urban sun france color tree gr...,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",8301,1,"0.104426,0.0598804,-0.0149883,0.0338535,-0.007...",0,49,1277859911.0,12,None,27597418605
304948,https://www.flickr.com/photos/92475605@N03/275...,27597517425.jpg,1,1.00,Social&People,portrait,1118334,108435@N35,Selfies,portrait cute animal cat furry panasonic calic...,...,NaN,801,0,"0.114495,-0.0303421,-0.0893819,0.0774031,-0.08...",0,3,1398624531.0,-6,matty3126,27597517425
304949,https://www.flickr.com/photos/43770052@N08/275...,27597606545.jpg,1,2.00,Travel&Active&Sports,exotic,1118342,84730@N71,Cars,car exotic german malaysia kualalumpur superca...,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",574,0,"0.18969,-0.0675743,-0.125082,0.097416,-0.02100...",0,2,1394377573.0,0,khooxotics,27597606545
304950,https://www.flickr.com/photos/50969511@N02/275...,27597637235.jpg,1,6.89,Whether&Season,cloud,1118344,9542@N37,Cloudy,city newzealand christchurch cloud building ca...,...,NaN,7167,1,"0.142518,-0.0121231,-0.0464556,0.0887664,-0.02...",0,20,1234576422.0,10,stevetaylors,27597637235


In [13]:
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
import requests

# Assuming you have a DataFrame dfraw with an 'imgid' column
# Example:
# dfraw = pd.DataFrame({
#     'imgid': ['385070026', '385070027', '385070028']  # Example image IDs
# })

# Function to get photo groups
def get_photo_groups(api_key, photo_id, per_page=200):
    url = "https://api.flickr.com/services/rest"
    params = {
        "method": "flickr.photos.getGroups",
        "api_key": api_key,
        "photo_id": photo_id,
        "format": "json",
        "nojsoncallback": "1",
        "per_page": str(per_page)
    }
    
    response = requests.get(url, params=params)
    groups = []
    if response.status_code == 200:
        data = response.json()
        groups = data.get("groups", {}).get("group", [])
    else:
        print("Error:", response.status_code)
    
    return groups

# Function for safe parallel execution
def get_photo_groups_safe(api_key, photo_id, per_page=200):
    try:
        return get_photo_groups(api_key, photo_id, per_page)
    except Exception as e:
        print(f"Error with image ID {photo_id}: {e}")
        return []  # Return an empty list in case of an error

In [14]:
# API Key
api_key = "169307181f957a4a16e4e30fc9ac8c77"  # Replace with your actual API key

# List of image IDs to process
imgids = df['imgID'].tolist()

# Set up parallel execution
parallel_results = Parallel(n_jobs=-1)(
    delayed(get_photo_groups_safe)(api_key, imgid) for imgid in tqdm(imgids, desc="Processing images")
)

Processing images:   0%|          | 0/251295 [00:00<?, ?it/s]

Processing images:   4%|▍         | 10856/251295 [04:54<9:40:45,  6.90it/s]

Error with image ID 17481904370: HTTPSConnectionPool(host='api.flickr.com', port=443): Max retries exceeded with url: /services/rest?method=flickr.photos.getGroups&api_key=169307181f957a4a16e4e30fc9ac8c77&photo_id=17481904370&format=json&nojsoncallback=1&per_page=200 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10301efb0>, 'Connection to api.flickr.com timed out. (connect timeout=None)'))
Error with image ID 17482841371: HTTPSConnectionPool(host='api.flickr.com', port=443): Max retries exceeded with url: /services/rest?method=flickr.photos.getGroups&api_key=169307181f957a4a16e4e30fc9ac8c77&photo_id=17482841371&format=json&nojsoncallback=1&per_page=200 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1064ffa30>, 'Connection to api.flickr.com timed out. (connect timeout=None)'))


Processing images:   4%|▍         | 10864/251295 [05:54<158:20:27,  2.37s/it]

Error with image ID 17488081072: HTTPSConnectionPool(host='api.flickr.com', port=443): Max retries exceeded with url: /services/rest?method=flickr.photos.getGroups&api_key=169307181f957a4a16e4e30fc9ac8c77&photo_id=17488081072&format=json&nojsoncallback=1&per_page=200 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x105576c50>, 'Connection to api.flickr.com timed out. (connect timeout=None)'))


...

Processing images:  24%|██▍       | 61152/251295 [40:13<1:22:11, 38.56it/s]

Error with image ID 21777623493: HTTPSConnectionPool(host='api.flickr.com', port=443): Max retries exceeded with url: /services/rest?method=flickr.photos.getGroups&api_key=169307181f957a4a16e4e30fc9ac8c77&photo_id=21777623493&format=json&nojsoncallback=1&per_page=200 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x10770f6a0>, 'Connection to api.flickr.com timed out. (connect timeout=None)'))


Processing images:  25%|██▌       | 63048/251295 [41:03<1:29:04, 35.22it/s]

KeyboardInterrupt: 

Processing images:  25%|██▌       | 63048/251295 [41:21<1:29:04, 35.22it/s]

In [15]:
parallel_results

NameError: name 'parallel_results' is not defined

In [16]:
import pandas as pd
import requests

# Assuming dfraw is your existing DataFrame with an 'imgid' column
# Example:
# dfraw = pd.DataFrame({
#     'imgid': ['385070026', '385070027', '385070028']  # Example image IDs
# })

# Function to get photo groups
def get_photo_groups(api_key, photo_id, per_page=200):
    url = "https://api.flickr.com/services/rest"
    params = {
        "method": "flickr.photos.getGroups",
        "api_key": api_key,
        "photo_id": photo_id,
        "format": "json",
        "nojsoncallback": "1",
        "per_page": str(per_page)
    }
    
    response = requests.get(url, params=params)
    groups = []
    if response.status_code == 200:
        data = response.json()
        groups = data.get("groups", {}).get("group", [])
    else:
        print("Error:", response.status_code)
    
    return groups

# API Key
api_key = "169307181f957a4a16e4e30fc9ac8c77"  # Replace with your actual API key

# Initialize an empty list to store the group information
group_data = []

In [17]:
# Iterate over each image ID in dfraw
for imgid in df['imgID'][:100]:
    # Get groups for the current image ID
    groups = get_photo_groups(api_key, imgid)

    # Iterate over each group and store the details along with the image ID
    for group in groups:
        group_info = {
            "imgid": imgid,
            "group_id": group.get("id"),
            "title": group.get("title"),
            "member_count": group.get("member_count"),
            "pool_count": group.get("pool_count")
            # Add other details here if needed
        }
        group_data.append(group_info)

# Create a DataFrame from the collected group data
group_df = pd.DataFrame(group_data)

# Display the first few rows of the new DataFrame
print(group_df.head())

       imgid      group_id                                           title  \
0  385070026   2131946@N24   ♥♥♥PE0PLE FR0M AR0UND THE W0RLD♥♥♥"FRIENDS"♥♥   
1  385070026   1344524@N24             Viva la Vida!  Alma e Cuore (P1-A5)   
2  385070026    855034@N21  The Biggest Group! - Playground For Psychotics   
3  385070026  75277045@N00             Self -help group for flickr addicts   
4  385070026  26964488@N00                              Powerful Portraits   

  member_count pool_count  
0         1115       6875  
1         9463     340173  
2         8228    1265747  
3          785      20454  
4        10666     330500  


In [19]:
group_df['imgid'].value_counts()

4497890817    154
4604352563     48
385070026      34
5960225601     21
5491588399      8
943750056       6
3544891702      2
4750078414      2
3538960847      1
3544037429      1
Name: imgid, dtype: int64

# Run this for your range

In [21]:
# Iterate over each image ID in dfraw
for imgid in tqdm(df['imgID'][100:1000], desc="Processing image IDs"):
    # Get groups for the current image ID
    groups = get_photo_groups(api_key, imgid)

    # Iterate over each group and store the details along with the image ID
    for group in groups:
        group_info = {
            "imgid": imgid,
            "group_id": group.get("id"),
            "title": group.get("title"),
            "member_count": group.get("member_count"),
            "pool_count": group.get("pool_count")
            # Add other details here if needed
        }
        group_data.append(group_info)

278
279
280


281
282
283
284
285


286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359


360
361
362
363
364


365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400


...

4858
4859
4860
4861
4862
4863
4864
4865
4866
4867
4868
4869
4870
4871
4872
4873
4874
4875
4876
4877
4878
4879
4880


4881
4882
4883
4884
4885
4886


4887
4888
4889


4890
4891
4892
4893
4894
4895
4896
4897
4898
4899
4900
4901
4902
4903
4904
4905
4906
4907
4908
4909
4910
4911
4912
4913
4914
4915


Processing image IDs: 100%|██████████| 900/900 [03:10<00:00,  4.71it/s]


In [25]:
# Create a DataFrame from the collected group data
group_df = pd.DataFrame(group_data)

# Display the first few rows of the new DataFrame
group_df

,imgid,group_id,title,member_count,pool_count
0,385070026,2131946@N24,"♥♥♥PE0PLE FR0M AR0UND THE W0RLD♥♥♥""FRIENDS""♥♥",1115,6875
1,385070026,1344524@N24,Viva la Vida! Alma e Cuore (P1-A5),9463,340173
2,385070026,855034@N21,The Biggest Group! - Playground For Psychotics,8228,1265747
3,385070026,75277045@N00,Self -help group for flickr addicts,785,20454
4,385070026,26964488@N00,Powerful Portraits,10666,330500
...,...,...,...,...,...
4910,16272277884,992994@N21,50mm for fiddy days,44,599
4911,16272277884,873614@N24,Canon EF 50mm 1:1.8 II,7423,81793
4912,16272277884,38788700@N00,"+ Sneakers ( 鞋子,スニーカー )",1567,19632
4913,16272277884,26773844@N00,Air Jordan,839,4080


In [26]:
group_df['imgid'].value_counts()

4497890817     154
12288905253    153
16252707364    119
16249237884    110
13928613034    110
              ... 
16199971983      1
15463930689      1
15465000670      1
16197777543      1
15422857619      1
Name: imgid, Length: 286, dtype: int64

In [27]:
import time
from tqdm import tqdm
# df, start_index, end_index, api_key, 
rate_limit=2000

# Calculate the time interval between requests to respect the rate limit
interval = 60.0 / rate_limit  # 60 seconds divided by the number of requests per minute

last_request_time = time.time()

# Wrap the iteration in tqdm for progress tracking
for imgid in tqdm(df['imgID'][1000:11000], desc="Processing image IDs", position=0):
    try:
        # Wait if the interval since the last request is less than the desired rate limit interval
        while time.time() - last_request_time < interval:
            time.sleep(interval - (time.time() - last_request_time))

        # Get groups for the current image ID
        groups = get_photo_groups(api_key, imgid)
        last_request_time = time.time()

        # Iterate over each group and store the details
        for group in groups:
            group_info = {
                "imgid": imgid,
                "group_id": group.get("id"),
                "title": group.get("title"),
                "member_count": group.get("member_count"),
                "pool_count": group.get("pool_count")
            }
            group_data.append(group_info)

    except Exception as e:
        print(f"Error processing image ID {imgid}: {e}")
        # Optional: wait for 30 seconds before continuing
        time.sleep(30)

Processing image IDs:  32%|███▏      | 3203/10000 [12:59<6:07:35,  3.24s/it]

Error: 500


Processing image IDs:  62%|██████▏   | 6238/10000 [25:42<3:24:56,  3.27s/it]

Error: 500


Processing image IDs: 100%|██████████| 10000/10000 [41:07<00:00,  4.05it/s] 


In [28]:
# Create a DataFrame from the collected group data
group_df = pd.DataFrame(group_data)

# Display the first few rows of the new DataFrame
group_df

,imgid,group_id,title,member_count,pool_count
0,385070026,2131946@N24,"♥♥♥PE0PLE FR0M AR0UND THE W0RLD♥♥♥""FRIENDS""♥♥",1115,6875
1,385070026,1344524@N24,Viva la Vida! Alma e Cuore (P1-A5),9463,340173
2,385070026,855034@N21,The Biggest Group! - Playground For Psychotics,8228,1265747
3,385070026,75277045@N00,Self -help group for flickr addicts,785,20454
4,385070026,26964488@N00,Powerful Portraits,10666,330500
...,...,...,...,...,...
57547,17562316500,394469@N21,pet central,3810,37131
57548,17562316500,616311@N21,"Dogs, all shapes and sizes",7135,104010
57549,17562316500,35034344814@N01,Dogs! Dogs! Dogs!,128289,1068253
57550,17562316500,68596255@N00,Dogs for fun Photos :-),8760,110033


In [30]:
group_df.drop_duplicates()

,imgid,group_id,title,member_count,pool_count
0,385070026,2131946@N24,"♥♥♥PE0PLE FR0M AR0UND THE W0RLD♥♥♥""FRIENDS""♥♥",1115,6875
1,385070026,1344524@N24,Viva la Vida! Alma e Cuore (P1-A5),9463,340173
2,385070026,855034@N21,The Biggest Group! - Playground For Psychotics,8228,1265747
3,385070026,75277045@N00,Self -help group for flickr addicts,785,20454
4,385070026,26964488@N00,Powerful Portraits,10666,330500
...,...,...,...,...,...
57547,17562316500,394469@N21,pet central,3810,37131
57548,17562316500,616311@N21,"Dogs, all shapes and sizes",7135,104010
57549,17562316500,35034344814@N01,Dogs! Dogs! Dogs!,128289,1068253
57550,17562316500,68596255@N00,Dogs for fun Photos :-),8760,110033


In [44]:
group_df[(group_df['imgid']=='16945993822') & (group_df['title'].str.startswith('BLACK'))].sort_values(by='pool_count')

,imgid,group_id,title,member_count,pool_count
31750,16945993822,38613568@N00,BLACK AND WHITE PHOTO HEAVEN!,28631,1353242


In [29]:
group_df['imgid'].value_counts()

16945993822    690
16759614658    671
16697363559    301
16530370843    270
16546857483    260
              ... 
17086801125      1
16607328724      1
17212550712      1
16731734548      1
16503461338      1
Name: imgid, Length: 3327, dtype: int64

In [47]:
df.reset_index(inplace=True, drop=True)
df[df['imgID']=='17562316500']

,url,img,img_access,label,Category,Concept,Pid,Uid,Subcategory,Alltags,...,location_description,photo_count,ispro,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,Pathalias,imgID
10996,https://www.flickr.com/photos/127445042@N03/17...,17562316500.jpg,1,8.38,Animal,lovedogs,172479,19811@N93,Dogs,dog happy collie cross ears spaniel lovemydog ...,...,NaN,45862,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1,2,1152740661.0,0,None,17562316500


In [49]:
group_df.to_csv('group_meta.csv', index=False)

In [50]:
import time
from tqdm import tqdm
# df, start_index, end_index, api_key, 
rate_limit=2000

# Calculate the time interval between requests to respect the rate limit
interval = 60.0 / rate_limit  # 60 seconds divided by the number of requests per minute

last_request_time = time.time()

# Wrap the iteration in tqdm for progress tracking
for imgid in tqdm(df['imgID'][11000:50000], desc="Processing image IDs", position=0):
    try:
        # Wait if the interval since the last request is less than the desired rate limit interval
        while time.time() - last_request_time < interval:
            time.sleep(interval - (time.time() - last_request_time))

        # Get groups for the current image ID
        groups = get_photo_groups(api_key, imgid)
        last_request_time = time.time()

        # Iterate over each group and store the details
        for group in groups:
            group_info = {
                "imgid": imgid,
                "group_id": group.get("id"),
                "title": group.get("title"),
                "member_count": group.get("member_count"),
                "pool_count": group.get("pool_count")
            }
            group_data.append(group_info)

    except Exception as e:
        print(f"Error processing image ID {imgid}: {e}")
        # Optional: wait for 30 seconds before continuing
        time.sleep(30)

Processing image IDs:  17%|█▋        | 6733/39000 [27:01<1:52:49,  4.77it/s] 

Error processing image ID 18596030512: HTTPSConnectionPool(host='api.flickr.com', port=443): Max retries exceeded with url: /services/rest?method=flickr.photos.getGroups&api_key=169307181f957a4a16e4e30fc9ac8c77&photo_id=18596030512&format=json&nojsoncallback=1&per_page=200 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x2b49ca020>, 'Connection to api.flickr.com timed out. (connect timeout=None)'))


Processing image IDs:  17%|█▋        | 6791/39000 [28:02<1:59:49,  4.48it/s]  

Error processing image ID 18600714555: HTTPSConnectionPool(host='api.flickr.com', port=443): Max retries exceeded with url: /services/rest?method=flickr.photos.getGroups&api_key=169307181f957a4a16e4e30fc9ac8c77&photo_id=18600714555&format=json&nojsoncallback=1&per_page=200 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x28392c2e0>, 'Connection to api.flickr.com timed out. (connect timeout=None)'))


Processing image IDs:  18%|█▊        | 7107/39000 [30:03<2:04:28,  4.27it/s]  

...

Error: 500


Processing image IDs:  97%|█████████▋| 37763/39000 [2:54:23<30:35,  1.48s/it]  

Error: 500


Processing image IDs:  98%|█████████▊| 38029/39000 [4:49:15<07:23,  2.19it/s]


KeyboardInterrupt: 

In [53]:
import time
from tqdm import tqdm
# df, start_index, end_index, api_key, 
rate_limit=2000

# Calculate the time interval between requests to respect the rate limit
interval = 60.0 / rate_limit  # 60 seconds divided by the number of requests per minute

last_request_time = time.time()

# Wrap the iteration in tqdm for progress tracking
for imgid in tqdm(df['imgID'][48030:50000], desc="Processing image IDs", position=0):
    try:
        # Wait if the interval since the last request is less than the desired rate limit interval
        while time.time() - last_request_time < interval:
            time.sleep(interval - (time.time() - last_request_time))

        # Get groups for the current image ID
        groups = get_photo_groups(api_key, imgid)
        last_request_time = time.time()

        # Iterate over each group and store the details
        for group in groups:
            group_info = {
                "imgid": imgid,
                "group_id": group.get("id"),
                "title": group.get("title"),
                "member_count": group.get("member_count"),
                "pool_count": group.get("pool_count")
            }
            group_data.append(group_info)

    except Exception as e:
        print(f"Error processing image ID {imgid}: {e}")
        # Optional: wait for 30 seconds before continuing
        time.sleep(30)

Processing image IDs: 100%|██████████| 1970/1970 [07:52<00:00,  4.17it/s]


In [58]:
# Create a DataFrame from the collected group data
group_df = pd.DataFrame(group_data)

# Display the first few rows of the new DataFrame
group_df

,imgid,group_id,title,member_count,pool_count
0,385070026,2131946@N24,"♥♥♥PE0PLE FR0M AR0UND THE W0RLD♥♥♥""FRIENDS""♥♥",1115,6875
1,385070026,1344524@N24,Viva la Vida! Alma e Cuore (P1-A5),9463,340173
2,385070026,855034@N21,The Biggest Group! - Playground For Psychotics,8228,1265747
3,385070026,75277045@N00,Self -help group for flickr addicts,785,20454
4,385070026,26964488@N00,Powerful Portraits,10666,330500
...,...,...,...,...,...
243387,21240245799,19156427@N00,The Flikr Group,7207,1359625
243388,21240245799,1876700@N25,Picturesque New England,131,7976
243389,21240245799,583086@N22,old & vintage,2804,46038
243390,21240245799,95014477@N00,NIKON PROFESSIONAL,36559,1665189


In [57]:
len(group_data)

243392

In [60]:
group_df.drop_duplicates(inplace=True)

In [61]:
group_df.imgid.value_counts()

19347042068    806
16945993822    690
16759614658    671
21151577326    645
20016125092    638
              ... 
19914861806      1
19914872586      1
19914902606      1
19914904406      1
19144826126      1
Name: imgid, Length: 12302, dtype: int64

In [62]:
12302/50000

0.24604

In [63]:
group_df.imgid.value_counts().describe()

count    12302.000000
mean        19.602179
std         39.359428
min          1.000000
25%          3.000000
50%          7.000000
75%         21.000000
max        806.000000
Name: imgid, dtype: float64

In [68]:
df[df['imgID']=='20016125092'].columns

Index(['url', 'img', 'img_access', 'label', 'Category', 'Concept', 'Pid',
       'Uid', 'Subcategory', 'Alltags', 'Title', 'Postdate', 'Longitude',
       'Geoaccuracy', 'Latitude', 'photo_firstdatetaken',
       'location_description', 'photo_count', 'ispro', 'user_description',
       'canbuypro', 'timezone_timezone_id', 'photo_firstdate',
       'timezone_offset', 'Pathalias', 'imgID'],
      dtype='object')

In [69]:
df[df['imgID']=='20016125092'].to_dict()

{'url': {34652: 'https://www.flickr.com/photos/87174189@N06/20016125092'},
 'img': {34652: '20016125092.jpg'},
 'img_access': {34652: 1},
 'label': {34652: 14.76},
 'Category': {34652: 'Family'},
 'Concept': {34652: 'cuddle'},
 'Pid': {34652: 206289},
 'Uid': {34652: '29877@N46'},
 'Subcategory': {34652: 'Babies'},
 'Alltags': {34652: 'camera morning flowers light sunset portrait people color guy girl beauty grass backlight sunrise canon eos eyes focus kiss soft closed flickr all chica view outdoor mark flash philips uptown wreath ii f cuddle ?? headphones 5d softfocus 28 clasp mm 135 sensuality scape embrace ef hotspot ease multa citi citiscape ?? ?? ??? ?? ?? ?? ??? ?? ???? ??? ?? ?? ???????? ???? ?? ??? ???? generosamente ??? ????135???????f 28???????? shl5905 kontrovoj'},
 'Title': {34652: 'Canon EF 135mm f / 2.8 SoftFocus'},
 'Postdate': {34652: 1436392782},
 'Longitude': {34652: 0.0},
 'Geoaccuracy': {34652: 0},
 'Latitude': {34652: 0.0},
 'photo_firstdatetaken': {34652: 19801004

In [71]:
df['label'].describe()

count    251295.000000
mean          6.358362
std           2.446125
min           1.000000
25%           4.700000
50%           6.190000
75%           7.900000
max          16.560000
Name: label, dtype: float64

In [72]:
df['label'].value_counts()

1.00     3477
1.58     3010
2.81     2933
3.00     2807
4.00     2792
         ... 
15.16       1
16.13       1
16.06       1
15.72       1
15.62       1
Name: label, Length: 1059, dtype: int64

In [75]:
group_df

,imgid,group_id,title,member_count,pool_count
0,385070026,2131946@N24,"♥♥♥PE0PLE FR0M AR0UND THE W0RLD♥♥♥""FRIENDS""♥♥",1115,6875
1,385070026,1344524@N24,Viva la Vida! Alma e Cuore (P1-A5),9463,340173
2,385070026,855034@N21,The Biggest Group! - Playground For Psychotics,8228,1265747
3,385070026,75277045@N00,Self -help group for flickr addicts,785,20454
4,385070026,26964488@N00,Powerful Portraits,10666,330500
...,...,...,...,...,...
243387,21240245799,19156427@N00,The Flikr Group,7207,1359625
243388,21240245799,1876700@N25,Picturesque New England,131,7976
243389,21240245799,583086@N22,old & vintage,2804,46038
243390,21240245799,95014477@N00,NIKON PROFESSIONAL,36559,1665189


In [76]:
group_df['imgid'].value_counts()

19347042068    806
16945993822    690
16759614658    671
21151577326    645
20016125092    638
              ... 
19914861806      1
19914872586      1
19914902606      1
19914904406      1
19144826126      1
Name: imgid, Length: 12302, dtype: int64

In [77]:
group_df['group_id'].value_counts()

76535076@N00       1065
95309787@N00        932
43501458@N00        669
81127001@N00        640
52240293230@N01     631
                   ... 
2872030@N22           1
528827@N23            1
36535838@N00          1
948065@N25            1
1876700@N25           1
Name: group_id, Length: 29560, dtype: int64

In [78]:
group_df[group_df['group_id']=='76535076@N00']

,imgid,group_id,title,member_count,pool_count
515,6982881973,76535076@N00,Flickr Addicts,138259,9041713
560,6988048739,76535076@N00,Flickr Addicts,138259,9041713
683,7742715506,76535076@N00,Flickr Addicts,138259,9041713
790,7916629758,76535076@N00,Flickr Addicts,138259,9041713
823,7980879218,76535076@N00,Flickr Addicts,138259,9041713
...,...,...,...,...,...
242651,21227854340,76535076@N00,Flickr Addicts,138258,9041710
242879,21233423846,76535076@N00,Flickr Addicts,138258,9041710
243137,21236896276,76535076@N00,Flickr Addicts,138258,9041710
243266,21239159703,76535076@N00,Flickr Addicts,138258,9041710


In [80]:
df[df['imgID']=='6982881973'].to_dict()

{'url': {116: 'https://www.flickr.com/photos/28991435@N02/6982881973'},
 'img': {116: '6982881973.jpg'},
 'img_access': {116: 1},
 'label': {116: 14.35},
 'Category': {116: 'Urban'},
 'Concept': {116: 'artwork'},
 'Pid': {116: 48688},
 'Uid': {116: '4352@N69'},
 'Subcategory': {116: 'StreetArt'},
 'Alltags': {116: 'door art museum painting sketch artwork diptych artist hand candid kunst eingang workinprogress painter tor wald farbe veduta innenraum absperrung hades perspektive malen untergrund tangram handwerk kunstwerk tre vedute aquarell knstler winterlandschaft malerei maler pinsel wintermrchen diptychon kreide gemlde skizze verzerrung durchgang zugang ungestellt gemldegalerie kunsthistorischesmuseumwien sockel vorort unterwelt wienvienna mischtechnik sterreichaustria dreidimensional grundierung parkettboden totenreich museumoffineartsvienna lokulus gnterfritsch vedutenmaler uninszeniert gysbrechtlytens amschauplatz'},
 'Title': {116: 'Painting at the Museum of Fine Arts Vienna - Gn

In [81]:
df['user_description'].value_counts()

0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [84]:
group_df.to_csv('group_meta_0_to_50000.csv', index=False)